In [ ]:
!pip install git+https://github.com/jpata/gym-so100.git@integration
!pip install imageio torch

In [3]:
import imageio
import gymnasium as gym
import numpy as np
import gym_so100
import numpy
#import torch

In [4]:
env = gym.make("gym_so100/PushCube-v0", render_mode="human")
observation, info = env.reset()

frames = []

/home/absin/Documents/gym-so100/venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65548) b'Wayland: The platform does not provide the window position'
  warnings.warn(message, GLFWError)


In [9]:
observation

{'agent_pos': array([ 0.  , -3.14,  3.  ,  1.24,  0.  ,  0.  ], dtype=float32),
 'agent_vel': array([0., 0., 0., 0., 0., 0.], dtype=float32),
 'target_pos': array([ 0.01823854, -0.18954605,  0.01      ], dtype=float32),
 'image_front': array([[[39, 66, 92],
         [39, 66, 92],
         [39, 66, 92],
         ...,
         [39, 66, 92],
         [39, 66, 92],
         [39, 66, 92]],
 
        [[39, 66, 92],
         [39, 66, 92],
         [39, 66, 92],
         ...,
         [39, 66, 92],
         [39, 66, 92],
         [39, 66, 92]],
 
        [[39, 65, 92],
         [39, 65, 92],
         [39, 65, 92],
         ...,
         [39, 65, 92],
         [39, 65, 92],
         [39, 65, 92]],
 
        ...,
 
        [[29, 59, 87],
         [29, 59, 87],
         [29, 59, 87],
         ...,
         [29, 59, 88],
         [29, 59, 88],
         [29, 59, 88]],
 
        [[29, 59, 87],
         [29, 59, 87],
         [29, 59, 87],
         ...,
         [29, 59, 88],
         [29, 59, 88],
 

In [15]:
env.action_space

Box(-1.0, 1.0, (6,), float32)

In [16]:
env.unwrapped.model

In [7]:
numpy_observation, reward, terminated, truncated, info = env.step(np.array([+5,0,0,0,0,0]))

/home/absin/Documents/gym-so100/venv/lib/python3.12/site-packages/gymnasium/utils/passive_env_checker.py:158: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


In [14]:
info

{'is_success': np.False_}

In [4]:
from pynput.keyboard import Key, Listener


In [5]:
def show(key):
    if key == Key.delete:
        # Stop listener
        return False
    
    if key.char == 'q':
        env.step(np.array([-5,0,0,0,0,0]))
    if key.char == 'w':
        env.step(np.array([5,0,0,0,0,0]))

    if key.char == 'a':
        env.step(np.array([0,5,0,0,0,0]))
    if key.char == 's':
        env.step(np.array([0,-5,0,0,0,0]))

    if key.char == 'z':
        env.step(np.array([0,0,5,0,0,0]))
    if key.char == 'x':
        env.step(np.array([0,0,-5,0,0,0]))

    if key.char == 'o':
        env.step(np.array([0,0,0,-5,0,0]))
    if key.char == 'p':
        env.step(np.array([0,0,0,5,0,0]))

    if key.char == 'k':
        env.step(np.array([0,0,0,0,-5,0]))
    if key.char == 'l':
        env.step(np.array([0,0,0,0,5,0]))

    if key.char == 'n':
        env.step(np.array([0,0,0,0,0,-5]))
    if key.char == 'm':
        env.step(np.array([0,0,0,0,0,5]))
 

In [6]:

# Collect all event  released
with Listener(on_press = show) as listener:   
   listener.join()

/home/absin/Documents/gym-so100/venv/lib/python3.12/site-packages/glfw/__init__.py:917: GLFWError: (65544) b'EGL: Failed to make context current: EGL cannot access a requested resource'
  warnings.warn(message, GLFWError)


In [7]:
env.close()

## 

In [ ]:
kk

In [ ]:
import pygame


In [ ]:
from lerobot.common.policies.act.modeling_act import ACTPolicy
pretrained_path = "/home/absin/Documents/git/robotics/act_algorithm/trained_model/act_aloha_sim_transfer_cube_human"
policy = ACTPolicy.from_pretrained(pretrained_path)
policy.reset()
numpy_observation, info = env.reset(seed=42)
rewards = []
frames = []
# Render frame of the initial state
frames.append(env.render())
step = 0
done = False
device = 'cpu'
while not done:
    # Prepare observation for the policy running in Pytorch
    state = torch.from_numpy(numpy_observation["agent_pos"])
    image = torch.from_numpy(numpy_observation["image_top"])
    state = torch.zeros(14)
    state[:numpy_observation["agent_pos"].shape[0]] = torch.from_numpy(numpy_observation["agent_pos"])
    # Convert to float32 with image from channel first in [0,255]
    # to channel last in [0,1]
    state = state.to(torch.float32)
    image = image.to(torch.float32) / 255
    image = image.permute(2, 0, 1)

    # Send data tensors from CPU to GPU
    state = state.to(device, non_blocking=True)
    image = image.to(device, non_blocking=True)

    # Add extra (empty) batch dimension, required to forward the policy
    state = state.unsqueeze(0)
    image = image.unsqueeze(0)

    # Create the policy input dictionary
    observation = {
        "observation.state": state,
        "observation.images.top": image,
    }

    # Predict the next action with respect to the current observation
    with torch.inference_mode():
        action = policy.select_action(observation)

    # Prepare the action for the environment
    numpy_action = action.squeeze(0).to("cpu").numpy()

    # Step through the environment and receive a new observation
    numpy_observation, reward, terminated, truncated, info = env.step(numpy_action[:6])
    print(f"{step=} {reward=} {terminated=}")

    # Keep track of all the rewards and frames
    rewards.append(reward)
    frames.append(env.render())

    # The rollout is considered done when the success state is reach (i.e. terminated is True),
    # or the maximum number of iterations is reached (i.e. truncated is True)
    done = terminated | truncated | done
    step += 1

if terminated:
    print("Success!")
else:
    print("Failure!")

imageio.mimsave('5_so100.mp4', numpy.stack(frames), fps=25)
print(f"Video of the evaluation is available in 5_so100.mp4.")

In [ ]:
env.render()

In [ ]:
padded_agent_pos = torch.zeros(14)
padded_agent_pos[:numpy_observation["agent_pos"].shape[0]] = torch.from_numpy(numpy_observation["agent_pos"])


In [ ]:
padded_agent_pos.shape

In [ ]:
!pip uninstall torch